In [2]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.Bayescard_BN import Bayescard_BN
from Evaluation.utils import parse_query
from Evaluation.cardinality_estimation import parse_query_single_table

In [3]:
query = "SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0"

In [4]:
df = pd.read_csv("/home/ziniu.wzn/Census/Census.csv", header=0, sep=",")
df = df.drop("caseid", axis=1)
df = df.dropna(axis=0)
print(len(df))
print(df.columns)

KeyboardInterrupt: 

In [5]:
BN = Pgmpy_BN("census")
BN.build_from_data(df, algorithm="chow-liu", max_parents=1, ignore_cols=['id'], sample_size=30000)
model_path = '/home/ziniu.wzn/BN_checkpoints/check_points/Census_chow-liu.pkl'
pickle.dump(BN, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)

NameError: name 'Pgmpy_BN' is not defined

In [6]:
with open('/home/ziniu.wzn/BN_checkpoints/check_points/Census_chow-liu.pkl', 'rb') as f:
    BN = pickle.load(f)
#with open('check_points/Census_junction.pkl', 'rb') as f:
 #   BN_J = pickle.load(f)

ModuleNotFoundError: No module named 'Models.pgmpy_BN'

In [7]:
BN.model.nodes

NodeView(('iLang1', 'dAncstry1', 'dAncstry2', 'iLooking', 'iAvail', 'iRPOB', 'iCitizen', 'dIndustry', 'iClass', 'dTravtime', 'dDepart', 'iDisabl2', 'iDisabl1', 'iYearwrk', 'iEnglish', 'iRvetserv', 'iFeb55', 'iRelat1', 'iFertil', 'dHispanic', 'dRearning', 'dHour89', 'iMeans', 'dHours', 'iImmigr', 'dIncome1', 'dIncome2', 'dIncome3', 'dRpincome', 'dIncome4', 'dAge', 'dIncome5', 'dIncome6', 'dIncome7', 'dIncome8', 'dOccup', 'iKorean', 'iYearsch', 'iRlabor', 'iRspouse', 'iMarital', 'iMay75880', 'iMilitary', 'iMobility', 'iMobillim', 'iOthrserv', 'iPerscare', 'dPOB', 'dPoverty', 'dPwgt1', 'iRagechld', 'dWeek89', 'iRelat2', 'iRrelchld', 'iRemplpar', 'iRiders', 'iRownchld', 'iSchool', 'iSept80', 'iSex', 'iSubfam1', 'iSubfam2', 'iTmpabsnt', 'iVietnam', 'iWork89', 'iWorklwk', 'iWWII', 'dYrsserv'))

In [10]:
query = "SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0"
tab_query = parse_query_single_table(query, BN)
tab_query

{'iAvail': [0],
 'iClass': [0],
 'iKorean': [0, 1],
 'iMay75880': [0],
 'iRagechld': [4, 2, 0, 3, 1],
 'iRrelchld': [0]}

In [12]:
BN.infer_algo = "exact"
BN.init_inference_method()
cardinality_predict = BN.query(tab_query)

In [13]:
cardinality_predict

420933.0

In [ ]:
def convert_to_pandas_query(query):
    query_str = ""
    for attr in query:
        query_str += attr
        val = query[attr]
        if type(val) != list:
            query_str += (" == " + str(val) + " and ")
        elif len(val) == 1:
            query_str += (" == " + str(val[0]) + " and ")
        else:
            query_str += (" in " + str(val) + " and ")
    query_str = query_str[:-5]
    return query_str

In [ ]:
len(df.query(convert_to_pandas_query(tab_query)))

In [ ]:
from time import perf_counter
def evaluate_cardinality_sampling(BN, query_path, samp, ratio=0.01):
    # read all queries
    with open(query_path) as f:
        queries = f.readlines()
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):
        cardinality_true = int(query_str.split("||")[-1])
        query_str = query_str.split("||")[0]
        print(f"Predicting cardinality for query {query_no}: {query_str}")
        
        query = parse_query_single_table(query_str.strip(), BN)
        query = convert_to_pandas_query(query)
        card_start_t = perf_counter()
        cardinality_predict = len(samp.query(query))*(1/ratio)
        if cardinality_predict is None:
            continue
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif cardinality_predict == 0:
            cardinality_predict = 1
        elif cardinality_true == 0:
            cardinality_true = 1

        q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        print(f"latency: {latency_ms} and error: {q_error}")
        latencies.append(latency_ms)
        q_errors.append(q_error)
    return latencies, q_errors

In [21]:
from time import perf_counter
def evaluate_cardinality_single_table(model_path, query_path, infer_algo, sample_size=1000):
    # load BN
    with open(model_path, 'rb') as f:
        BN = pickle.load(f)
    if BN.infer_machine is None:
        print("okay")
        BN.infer_algo = infer_algo
        BN.init_inference_method()
    # read all queries
    with open(query_path) as f:
        queries = f.readlines()
    latencies = []
    q_errors = []
    for query_no, query_str in enumerate(queries):
        cardinality_true = int(query_str.split("||")[-1])
        query_str = query_str.split("||")[0]
        try:
            print(f"Predicting cardinality for query {query_no}: {query_str}")
            query = parse_query_single_table(query_str.strip(), BN)
            card_start_t = perf_counter()
            cardinality_predict = BN.query(query, sample_size=sample_size)
        except:
            #In the case, that the query itself is invalid or contains some values that are not recognizable by BN
            continue
        card_end_t = perf_counter()
        latency_ms = (card_end_t - card_start_t) * 1000
        if cardinality_predict == 0 and cardinality_true == 0:
            q_error = 1.0
        elif np.isnan(cardinality_predict) or cardinality_predict == 0:
            cardinality_predict = 1
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        elif cardinality_true == 0:
            cardinality_true = 1
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        else:
            q_error = max(cardinality_predict / cardinality_true, cardinality_true / cardinality_predict)
        print(f"latency: {latency_ms} and error: {q_error}")
        latencies.append(latency_ms)
        q_errors.append(q_error)

    print("=====================================================================================")
    for i in [50, 90, 95, 99, 100]:
        print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
    print(f"average latency is {np.mean(latencies)} ms")

    return latencies, q_errors

In [24]:
query_path = "/home/ziniu.wzn/Census/cardinality/query_one_side.sql"
latencies, q_errors = evaluate_cardinality_single_table('/home/ziniu.wzn/BN_checkpoints/check_points/Census_chow-liu.pkl', query_path, "progressive_sampling", 8000)

okay
Predicting cardinality for query 0: SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0
latency: 45.00521719455719 and error: 1.8102426789604253
Predicting cardinality for query 1: SELECT COUNT(*) FROM climate WHERE dIncome5 = 0 AND iOthrserv = 0 AND dPOB = 0 AND iRownchld = 0 AND iSex >= 0 AND iSex <= 1 AND dYrsserv = 0
latency: 42.96734184026718 and error: 1.1005792184870182
Predicting cardinality for query 2: SELECT COUNT(*) FROM climate WHERE dAncstry1 >= 1 AND dAncstry1 <= 11 AND iEnglish = 0 AND iMarital = 0 AND iPerscare = 2 AND iRagechld >= 0 AND iRagechld <= 4 AND dRearning = 0 AND iRspouse >= 1 AND iRspouse <= 3 AND iYearsch >= 0 AND iYearsch <= 10
latency: 42.30184108018875 and error: 1.1532436195285094
Predicting cardinality for query 3: SELECT COUNT(*) FROM climate WHERE dIncome2 = 0 AND dIncome7 = 0 AND dRearning >= 0 AND dRearning <= 3 AND iSex = 1 AN

latency: 43.57101768255234 and error: 1.0021764805617983
Predicting cardinality for query 30: SELECT COUNT(*) FROM climate WHERE dOccup >= 0 AND dOccup <= 5 AND dPwgt1 = 1 AND iRelat1 >= 0 AND iRelat1 <= 2 AND iYearwrk >= 0 AND iYearwrk <= 4
latency: 42.74123162031174 and error: 1.0203371207955043
Predicting cardinality for query 31: SELECT COUNT(*) FROM climate WHERE dDepart >= 0 AND dDepart <= 3 AND dIncome7 = 0 AND dIndustry = 0 AND iMarital >= 0 AND iMarital <= 4 AND iMobillim = 2 AND iSex = 1 AND iWWII = 0
latency: 43.951377272605896 and error: 1.2380327621987886
Predicting cardinality for query 32: SELECT COUNT(*) FROM climate WHERE iDisabl2 >= 0 AND iDisabl2 <= 2 AND dIncome4 = 0 AND dRearning >= 2 AND dRearning <= 3 AND iVietnam = 0
latency: 44.13549602031708 and error: 1.0716722659837068
Predicting cardinality for query 33: SELECT COUNT(*) FROM climate WHERE iDisabl2 = 2 AND iImmigr = 0 AND iMarital = 4 AND iMeans >= 1 AND iMeans <= 10 AND iMobility = 1 AND iRrelchld = 1 AND i

latency: 43.28005760908127 and error: 1.1320451975784502
Predicting cardinality for query 60: SELECT COUNT(*) FROM climate WHERE iDisabl2 = 0 AND dHours >= 0 AND dHours <= 4 AND iLooking >= 0 AND iLooking <= 2 AND iSept80 = 0 AND iSubfam2 = 0 AND iTmpabsnt >= 0 AND iTmpabsnt <= 3 AND iWWII = 0 AND iYearsch = 10
latency: 42.89659112691879 and error: 1053.7356926746152
Predicting cardinality for query 61: SELECT COUNT(*) FROM climate WHERE iClass >= 0 AND iClass <= 1 AND dIncome3 = 0 AND iRelat2 >= 0 AND iRelat2 <= 1 AND iRspouse = 1 AND iRvetserv = 0 AND dTravtime = 0 AND iVietnam = 0 AND iWork89 = 0
latency: 41.961804032325745 and error: 1975.9090441492076
Predicting cardinality for query 62: SELECT COUNT(*) FROM climate WHERE dAncstry2 = 1 AND dHispanic = 0 AND dHour89 >= 1 AND dHour89 <= 4 AND dHours = 0 AND dIncome3 = 0 AND dIncome6 = 0 AND iOthrserv = 0 AND iPerscare >= 1 AND iPerscare <= 2 AND iRagechld >= 0 AND iRagechld <= 4
latency: 42.45626926422119 and error: 2.34791684319567

latency: 45.521169900894165 and error: 1.081565090191442
Predicting cardinality for query 90: SELECT COUNT(*) FROM climate WHERE iCitizen >= 0 AND iCitizen <= 4 AND dDepart = 0 AND iDisabl2 >= 0 AND iDisabl2 <= 1 AND iImmigr >= 0 AND iImmigr <= 1 AND dIncome6 = 0 AND iMay75880 = 0 AND iMeans >= 0 AND iMeans <= 1 AND dOccup >= 0 AND dOccup <= 1 AND iRemplpar = 0 AND iWWII = 0
latency: 43.80679875612259 and error: 2.1414388561954545
Predicting cardinality for query 91: SELECT COUNT(*) FROM climate WHERE iCitizen >= 0 AND iCitizen <= 3 AND dIncome6 = 0 AND iRvetserv = 0 AND iSept80 = 0
latency: 44.88445073366165 and error: 1.0029696671441923
Predicting cardinality for query 92: SELECT COUNT(*) FROM climate WHERE iImmigr = 0 AND dIncome1 = 2 AND iMilitary = 4 AND iSept80 = 0
latency: 44.43831741809845 and error: 1.241496312331375
Predicting cardinality for query 93: SELECT COUNT(*) FROM climate WHERE iFeb55 = 0 AND dIncome5 = 0 AND dIncome6 >= 0 AND dIncome6 <= 1 AND iMeans >= 0 AND iMeans

latency: 44.070176780223846 and error: 1.6534115981540403
Predicting cardinality for query 120: SELECT COUNT(*) FROM climate WHERE iClass >= 0 AND iClass <= 6 AND dDepart >= 0 AND dDepart <= 2 AND iDisabl2 = 2 AND iFeb55 = 0 AND iFertil >= 0 AND iFertil <= 4 AND dHispanic = 0 AND iRelat2 = 0 AND iRiders >= 0 AND iRiders <= 2 AND iRvetserv = 0
latency: 43.14006119966507 and error: 1.3692566207008936
Predicting cardinality for query 121: SELECT COUNT(*) FROM climate WHERE iFeb55 = 0 AND dIndustry >= 5 AND dIndustry <= 9 AND iPerscare >= 0 AND iPerscare <= 2 AND iRownchld >= 0 AND iRownchld <= 1 AND iRvetserv = 0 AND iSubfam1 = 0 AND dWeek89 >= 1 AND dWeek89 <= 2 AND dYrsserv >= 0 AND dYrsserv <= 1
latency: 44.78594660758972 and error: 1.6972653261063009
Predicting cardinality for query 122: SELECT COUNT(*) FROM climate WHERE dAge >= 1 AND dAge <= 3 AND iEnglish = 0 AND dIncome1 = 0 AND dIncome5 = 0 AND iMilitary >= 0 AND iMilitary <= 4 AND iMobillim = 2 AND iRagechld = 0 AND iWork89 = 1


...

latency: 41.99604690074921 and error: 2.9411300034568715
Predicting cardinality for query 335: SELECT COUNT(*) FROM climate WHERE iClass = 0 AND iFeb55 >= 0 AND iFeb55 <= 1 AND dHispanic = 0 AND dIncome1 >= 0 AND dIncome1 <= 2 AND iLang1 >= 0 AND iLang1 <= 2 AND iRelat2 = 0 AND iRrelchld = 0 AND iRvetserv = 0 AND iSept80 = 0 AND iSubfam2 = 0
latency: 42.973652482032776 and error: 1.6599490555550775
Predicting cardinality for query 336: SELECT COUNT(*) FROM climate WHERE iDisabl1 >= 1 AND iDisabl1 <= 2 AND dIncome8 = 0 AND iMilitary = 4 AND dOccup = 0 AND iRemplpar >= 0 AND iRemplpar <= 222
latency: 43.293848633766174 and error: 1.2798530952434308
Predicting cardinality for query 337: SELECT COUNT(*) FROM climate WHERE dDepart = 0 AND iDisabl1 = 2 AND iFeb55 >= 0 AND iFeb55 <= 1 AND dHispanic = 0 AND dHours >= 0 AND dHours <= 1 AND iPerscare = 2 AND dPwgt1 >= 1 AND dPwgt1 <= 2 AND iRelat1 = 0 AND iWorklwk >= 0 AND iWorklwk <= 2
latency: 42.80322045087814 and error: 1.0912529768705468
Pr

latency: 44.170863926410675 and error: 1.01621202762841
Predicting cardinality for query 365: SELECT COUNT(*) FROM climate WHERE dDepart = 0 AND dIncome5 = 0 AND iMilitary = 4 AND iMobillim = 2 AND iSept80 = 0
latency: 43.14451664686203 and error: 1.3387954123870098
Predicting cardinality for query 366: SELECT COUNT(*) FROM climate WHERE dAncstry2 = 1 AND iFertil >= 3 AND iFertil <= 5 AND dIncome5 = 0 AND dIndustry >= 0 AND dIndustry <= 4 AND iKorean = 0 AND iMeans >= 0 AND iMeans <= 10 AND dOccup >= 2 AND dOccup <= 6 AND iYearsch >= 11 AND iYearsch <= 14
latency: 42.93020814657211 and error: 1.5518432483945228
Predicting cardinality for query 367: SELECT COUNT(*) FROM climate WHERE iDisabl1 = 2 AND iEnglish = 0 AND dIncome6 = 0 AND dIncome7 = 0 AND dIndustry >= 4 AND dIndustry <= 7 AND iRagechld = 4 AND iRPOB >= 23 AND iRPOB <= 52 AND iRrelchld = 0 AND iSubfam2 = 0 AND iWWII = 0
latency: 43.780699372291565 and error: 1.1079651878673995
Predicting cardinality for query 368: SELECT COUN

latency: 42.325615882873535 and error: 2.491348241991033
Predicting cardinality for query 395: SELECT COUNT(*) FROM climate WHERE dHour89 >= 0 AND dHour89 <= 2 AND dIncome2 = 0 AND dWeek89 >= 0 AND dWeek89 <= 1 AND iWork89 >= 0 AND iWork89 <= 1
latency: 43.63162815570831 and error: 1.3330369693427437
Predicting cardinality for query 396: SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass >= 0 AND iClass <= 1 AND iDisabl1 >= 0 AND iDisabl1 <= 2 AND iEnglish = 0 AND iRrelchld = 0 AND iWork89 = 1
latency: 43.11714321374893 and error: 1.0635798810709551
Predicting cardinality for query 397: SELECT COUNT(*) FROM climate WHERE dAncstry2 >= 1 AND dAncstry2 <= 3 AND dIncome2 = 0 AND iLang1 >= 1 AND iLang1 <= 2 AND iMeans = 0 AND iSex >= 0 AND iSex <= 1 AND iSubfam1 = 0 AND iSubfam2 = 0 AND iYearsch >= 3 AND iYearsch <= 11
latency: 41.50725156068802 and error: 1.0929116768994858
Predicting cardinality for query 398: SELECT COUNT(*) FROM climate WHERE dRearning >= 0 AND dRearning <= 4 AND 

latency: 43.81056874990463 and error: 1.0514993762789584
Predicting cardinality for query 430: SELECT COUNT(*) FROM climate WHERE dDepart >= 0 AND dDepart <= 4 AND iEnglish >= 0 AND iEnglish <= 4 AND dHispanic = 0 AND dIncome6 = 0 AND iMay75880 = 0 AND iMilitary = 0 AND iMobillim = 2 AND iRvetserv = 0 AND dWeek89 = 0 AND iWorklwk >= 0 AND iWorklwk <= 1
latency: 43.00723224878311 and error: 19.534329696369394
Predicting cardinality for query 431: SELECT COUNT(*) FROM climate WHERE iFeb55 = 0 AND dIncome7 = 0 AND iLang1 = 1 AND iOthrserv = 0
latency: 43.34608465433121 and error: 1.0318519937160486
Predicting cardinality for query 432: SELECT COUNT(*) FROM climate WHERE dIncome5 >= 0 AND dIncome5 <= 1 AND iKorean = 0 AND iMilitary >= 2 AND iMilitary <= 4 AND iRelat2 = 0 AND iRownchld >= 0 AND iRownchld <= 1 AND iRrelchld = 0 AND iSchool = 1
latency: 42.82967001199722 and error: 1.2971077761758634
Predicting cardinality for query 433: SELECT COUNT(*) FROM climate WHERE iClass >= 0 AND iCla

latency: 45.03008723258972 and error: 1.1364157793808907
Predicting cardinality for query 460: SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iCitizen = 0 AND dHour89 >= 2 AND dHour89 <= 5 AND dIncome1 >= 0 AND dIncome1 <= 3 AND dIncome7 = 0 AND iMarital >= 0 AND iMarital <= 4 AND dPOB = 0 AND dPwgt1 >= 1 AND dPwgt1 <= 3 AND iRelat1 >= 0 AND iRelat1 <= 1 AND iRspouse >= 0 AND iRspouse <= 1 AND iWWII = 0
latency: 43.36763918399811 and error: 1.2707891748337155
Predicting cardinality for query 461: SELECT COUNT(*) FROM climate WHERE iEnglish = 0 AND dIncome7 = 0 AND iMobillim >= 1 AND iMobillim <= 2 AND dRearning = 0
latency: 43.50169003009796 and error: 1.7273743103486798
Predicting cardinality for query 462: SELECT COUNT(*) FROM climate WHERE iImmigr = 0 AND iMarital >= 1 AND iMarital <= 2 AND iMay75880 = 0 AND iRlabor = 1 AND iRrelchld >= 0 AND iRrelchld <= 1 AND iVietnam = 0 AND iWorklwk >= 0 AND iWorklwk <= 1 AND iWWII = 0
latency: 42.899616062641144 and error: 1.1336581385232334

In [22]:
print(np.mean(latencies))
for i in [50,90,95,99,100]:
    print(np.percentile(q_errors, i))

11.674584868626717
1.196448551917472
3.0415310510638256
8.344291700913734
40.883576091788136
4068.0114603622296


In [ ]:
query = "SELECT COUNT(*) FROM climate WHERE iAvail = 0 AND iClass = 0 AND iKorean >= 0 AND iKorean <= 1 AND iMay75880 = 0 AND iRagechld >= 0 AND iRagechld <= 4 AND iRrelchld = 0"
query = parse_query_single_table(query.strip(), BN)
print(query)
tic = time.time()
BN.query(query)
print(time.time() - tic)